In [4]:
import pandas as pd

train_file = r'C:\Users\Aditya Oak\Desktop\Kaggle Intermediate ML course\train.csv'
test_file = r'C:\Users\Aditya Oak\Desktop\Kaggle Intermediate ML course\test.csv'

X_train_full = pd.read_csv(train_file, index_col='Id')
X_test_full = pd.read_csv(test_file, index_col='Id')

#X_train_full.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Remove rows with missing target

In [5]:
from sklearn.model_selection import train_test_split

#Remove rows with missing target
X_train_full.dropna(axis=0, subset=['SalePrice'], inplace=True)

y = X_train_full.SalePrice

# Seperate target from predictors
X_train_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, use only numeric values for predictors, exclude object types
X = X_train_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

#Break off validation data from training
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [6]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


# Step 1: Preliminary investigation
## We se a few missing values in columns other than target. NExt step we get a better understanding of all missing values

In [13]:
print(X_train.shape)

missing_val_by_col = X_train.isnull().sum()

print(missing_val_by_col[missing_val_by_col>0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [14]:
# How many total rows in training data
num_rows = 1168

# How many columns in training data have missing values
num_cols_with_missing = 3

# How many missing entries are contained in all of training data
tot_missing = missing_val_by_col.sum()

In [12]:
"""
# Build a few models of Random Forest Regressor with varying parameters

from sklearn.ensemble import RandomForestRegressor

model_1 = RandomForestRegressor(n_estimators=50, random_state = 0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='mae', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1,model_2,model_3,model_4,model_5]
"""

In [22]:
# Build a function that will compare approaches

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train,y_train,X_valid,y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_valid)
    return mean_absolute_error(y_pred,y_valid)

Model  1  Score:  24015.492818003917
Model  2  Score:  23740.979228636657
Model  3  Score:  23528.78421232877
Model  4  Score:  23996.676789668687
Model  5  Score:  23706.672864217904


# Step 2: Dropping columns with missing values

In [26]:
# Get column names with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# drop columns with missing data in training and validation
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)



IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [27]:
# Fit model to training data
# Generate predictions based on best model

best_model.fit(X_train,y_train)
y_preds_best = best_model.predict(X)

output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': y_preds_best})

output.to_csv('submission.csv', index = False)

NameError: name 'X_test' is not defined